In [3]:
import PySimpleGUI as sg
import vlc
from time import sleep
import os
from sys import platform

#----------- TEST MEDIA -----------#
import test_data
song1 = test_data.song1 # test a local path
song2 = test_data.song2 # test a remote path 

#----------- SETUP -----------#
number_of_media = 4

# window_namer = {}
# for w in range(1,number_of_media):
#     window_namer['window'+str(w)] = str(w)
# locals().update(window_namer)


#----------- GUI -----------#
def layout(number):
    # can't use the same layout each time, but it works if func just takes an unused param
    return [
        [sg.Graph(canvas_size=(1280,720), graph_bottom_left=(0,0), graph_top_right=(0,0), key='-G-')],
        [sg.Button('Exit', size=(10,1), pad=(0,0))]]

# Each window needs a unique name, so we'll use a dictionary to store them

window_names = {}
for i in range(1, number_of_media + 1):
    window_names['window' + str(i)] = sg.Window(str(i), layout(i), finalize=True, location=(0,0))
    locals().update(window_names)




#----------- POSITIONING -----------#


# #----------- PLAYER SETUP -----------#
# # shorten this code will need as many as 8 players
# instance_list = []
# for i in range(1,number_of_media + 1):
#     instance_list.append(vlc.Instance())

# media_list = []

# player_list = []
# for i in range(1,number_of_media + 1):
#     player_list.append(instance_list[i].media_player_new())
#     media_list.append(instance_list[i].media_new(song1))
    
#     # player_list[i].set_hwnd(window[i]['-G-'].Widget.winfo_id())
#     player_list[i].set_hwnd(window1['-G-'].Widget.winfo_id())

#     player_list[i].set_media(media_list[i])

# #----------- MAIN LOOP -----------#
# # next: make sure media is on loop

# i, paused = 0, [False, False]

# while True:
#     window, event, values = sg.read_all_windows(timeout=1000)
#     for i in range(1,number_of_media):
#         player_list[i].play()
#         if (window[i] == sg.WIN_CLOSED) or (event == 'Exit'):
#             break
#     # for i in range(8):
#     #     window[i].close()

# #----------- CLEANUP -----------#
# for i in range(1,number_of_media):
#     player_list[i].stop()
#     player_list[i].release()
#     instance_list[i].release()
#     window[i].close()


# #----------- OLD CODE -----------#
